In [0]:
# Install Pytorch
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x579ce000 @  0x7f9626d4a2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
# Needed to get the gym environment working
!apt-get install swig3.0
!ln -s /usr/bin/swig3.0 /usr/bin/swig
!pip3 install box2d box2d-kengz

[autoreload of PIL.Image failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name 'py3'
]
[autoreload of PIL._binary failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name 'py3'
]


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig3.0
0 upgraded, 1 newly installed, 0 to remove and 5 not upgraded.
Need to get 1,094 kB of archives.
After this operation, 5,499 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Fetched 1,094 kB in 1s (1,135 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 22298 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Setting up swig3.0 (3.0.12-1) ...
    100% |████████████████████████████████| 430kB 6.4MB/s 
    100% |████████████████████████████████| 430kB 7.7MB/s 
  Running setup.py bdist_wheel for box2d ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

In [0]:
!pip install gym
!pip install ptan

    100% |████████████████████████████████| 1.5MB 7.8MB/s 
    100% |████████████████████████████████| 1.0MB 9.5MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/6c/3a/0e/b86dee98876bb56cdb482cc1f72201035e46d1baf69d10d028
Successfully built gym


In [0]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import numpy as np
import gym
import ptan

from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython import display
plt.style.use('seaborn')

from collections import deque

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

np.random.seed(1)

In [0]:
ENV = gym.make('LunarLander-v2')
NS = ENV.observation_space.shape[0]  
NA = 1
DEFAULT_PARAMS = {'bn':10, 'bs':256, 'gamma':0.5, 'ns':NS, 'na':NA}
LR = 1e-2

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [0]:
ENV.reset()

array([ 0.00367823,  1.4185607 ,  0.3725416 ,  0.33957517, -0.00425527,
       -0.08438619,  0.        ,  0.        ], dtype=float32)

## Create Random Agent

In [0]:
class RandomAgent():
    
    def __init__(self, env, clip_reward = False):
        self.env = env
        self.max_rounds = int(1e4)
        self.clip_reward = clip_reward
        self.n = 0  # Number of exploration rounds
        
    def randomAction(self):
        return self.env.action_space.sample()
    
    def pi(self, state):
        return self.randomAction()
    
    def playPol(self, save_frames = False, save_ext = 'tmp/experiment0/'):
        
        '''Plays a game from start to finish. '''
        
        reward = 0
        
        if save_frames:
            env = wrappers.Monitor(self.env, save_ext, force=True)
        else:
            env = self.env
            
        state= env.reset()
        
        self.history = deque()
        for i in range(self.max_rounds):
            if save_frames: env.render()
            action = self.pi(state)
            newstate, r, done, _ = self.env.step(action)
            self.history.append((state, action, r, done, newstate))
            if self.clip_reward:
                r = max(min(r, 1), -1)
            reward += r
            if done: break
                
        if save_frames: 
            env.close(); self.env.close()
        return reward

In [0]:
random_guy = RandomAgent(ENV)

In [0]:
random_guy.playPol()

-320.556853800019

## Create Neural Architecture

In [0]:
arch = nn.Sequential(nn.Linear(8, 50), 
                    nn.ReLU(), 
                    nn.BatchNorm1d(50), 
                    nn.Linear(50, 4) )           

In [0]:
state = torch.tensor(ENV.reset()[None])
arch.eval()
with torch.no_grad():
    print(arch(state).numpy()[0])

[-0.14583129  0.0443883  -0.16089794 -0.16389734]


## Define Agent

In [0]:
def softmax(x):
    ex = np.exp(x)
    return ex / np.sum(ex)

In [0]:
class Policy_Agent(RandomAgent):
    
    def __init__(self, env, arch):
        super().__init__(env)
        self.arch = arch
        
    def pi(self, state):
        return np.random.choice(4, 1, True, p = self.getProbs(state))[0]
        
    def getProbs(self, state):
        return softmax(self.getNetOutput(state))
    
    def getNetOutput(self, state):
        self.arch.eval()
        with torch.no_grad():
            inp = torch.tensor(state[None])
            return self.arch(inp).numpy()[0]

In [0]:
policy_guy = Policy_Agent(ENV, arch)

In [0]:
policy_guy.playPol()

-190.87872821638484

## Try out ptan

Never used this library before. Let's try it.

In [0]:
agent = ptan.agent.PolicyAgent(arch.cuda(), preprocessor=ptan.agent.float32_preprocessor, apply_softmax=True, device='cuda')

In [0]:
exp_source = ptan.experience.ExperienceSourceFirstLast(ENV, agent, gamma=DEFAULT_PARAMS['gamma'], steps_count=10)

In [0]:
next(iter(exp_source))

ExperienceFirstLast(state=array([ 0.00146408,  1.4121182 ,  0.14827986,  0.05325324, -0.00168971,
       -0.03358761,  0.        ,  0.        ], dtype=float32), action=3, reward=-1.310207550652947, last_state=array([ 0.0188241 ,  1.4006064 ,  0.19714774, -0.12224919, -0.0503949 ,
       -0.10318725,  0.        ,  0.        ], dtype=float32))

Okay, so it seems this library just feeds us the state, action, reward tuples we need.